In [ ]:
class GOST28147_89:
    def __init__(self):
        self._key = [0x12345678, 0xABCD1234, 0x5678EFEF, 0x98765432, 0xFEDCBA12, 0x11223344, 0xAABBCCDD, 0x55667788]
        self._s_box = (
            (3, 7, 1, 9, 0, 2, 4, 8, 6, 3, 11, 12, 13, 14, 15, 10),
            (12, 6, 11, 2, 10, 4, 7, 5, 9, 0, 1, 3, 8, 13, 14, 15),
            (7, 2, 5, 0, 11, 14, 15, 10, 9, 12, 8, 1, 3, 4, 6, 13),
            (8, 14, 7, 6, 2, 5, 1, 15, 4, 0, 11, 10, 13, 9, 12, 3),
            (15, 6, 9, 11, 3, 12, 4, 2, 10, 8, 1, 0, 7, 14, 4, 13),
            (12, 15, 0, 9, 4, 8, 13, 2, 11, 10, 7, 6, 5, 3, 14, 1),
            (14, 3, 1, 10, 7, 15, 13, 12, 0, 2, 11, 4, 8, 9, 6, 5),
            (6, 8, 15, 4, 10, 7, 12, 1, 2, 0, 14, 5, 11, 3, 9, 13),
        )

    def f(self, right, k_i):
        right = (right + k_i) & 0xFFFFFFFF
        right = self.substitution(right)
        output = (right << 11) & 0xFFFFFFFF
        return output

    def substitution(self, right):
        result = 0
        for i in range(8):
            result |= ((self._s_box[i][(right >> (4 * i)) & 0xf]) << (4 * i))
        return result

    def encryption_round(self, input_left, input_right, round_key):
        output_left = input_right
        output_right = input_left ^ self.f(input_right, round_key)
        return output_left, output_right

    def decryption_round(self, input_left, input_right, round_key):
        output_right = input_left
        output_left = input_right ^ self.f(input_left, round_key)
        return output_left, output_right

    def encrypt(self, block):
        left, right = block >> 32, block & 0xFFFFFFFF
        for i in range(32):
            k_i = self._key[i % 8] if i < 24 else self._key[7 - (i % 8)]
            left, right = self.encryption_round(left, right, k_i)
        return (left << 32) | right

    def decrypt(self, block):
        left, right = block >> 32, block & 0xFFFFFFFF
        for i in range(32):
            k_i = self._key[i] if i < 8 else self._key[7 - (i % 8)]
            left, right = self.decryption_round(left, right, k_i)
        return (left << 32) | right


def string_to_block(text):
    binary_list = [format(ord(char), '08b') for char in text]
    binary_string = ''.join(binary_list).ljust(64, '0')
    return int(binary_string, 2)


def block_to_string(block):
    binary_string = format(block, '064b')
    return ''.join([chr(int(binary_string[i:i + 8], 2)) for i in range(0, 64, 8)])


def encrypt_text(text):
    cipher = GOST28147_89()
    segments = [text[i:i + 8] for i in range(0, len(text), 8)]
    encrypted_text = ""
    for segment in segments:
        data = string_to_block(segment)
        encrypt = cipher.encrypt(data)
        encrypted_text += block_to_string(encrypt)
    return encrypted_text


def decrypt_text(encrypted_text):
    cipher = GOST28147_89()
    segments = [encrypted_text[i:i + 8] for i in range(0, len(encrypted_text), 8)]
    decrypted_text = ""
    for segment in segments:
        data = string_to_block(segment)
        decrypt = cipher.decrypt(data)
        decrypted_text += block_to_string(decrypt)
    return decrypted_text


if __name__ == '__main__':
    text = input("Enter text: ")
    encrypted_result = encrypt_text(text)
    print("Encrypted:", encrypted_result)

    decrypted_result = decrypt_text(encrypted_result)
    print("Decrypted:", decrypted_result)
